In [1]:
from keras.models import model_from_json
from keras.models import Model
import numpy as np
from keras.utils import plot_model


Using TensorFlow backend.


In [13]:
#loading function
def load_model(n):
    json_file = open('new_model_'+str(n)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    sentiment_model = model_from_json(loaded_model_json)
    # load weights into new model
    sentiment_model.load_weights("new_model_"+str(n)+".h5")
    print("Loaded model"+str(n)+" from disk")
    return sentiment_model

In [14]:
vocab_fn = "word_list.txt"
with open(vocab_fn, 'r') as vfn:
    index2word = vfn.read().split('\n')
index2word = index2word[:-1]
print(len(index2word),"words in vocab")

mat_fn = "weight.npy"
embedding_mat = np.load(mat_fn)
print(embedding_mat.shape,"embedding matrix")

1048576 words in vocab
(1048576, 100) embedding matrix


In [15]:
#add NULL (0) and UNK to our vocab
lookup_with_unk = {word:i+2 for i,word in enumerate(index2word)}
UNK_IND = 1

#add null and UNK vectors to our embedding matrix so it still lines up
embeddings_with_unk = np.zeros((embedding_mat.shape[0]+2, embedding_mat.shape[1]))
embeddings_with_unk[2:] = embedding_mat

In [16]:
label_dict_r = {
    0:"1F44C",
    1:"1F44D",
    2:"1F60A",
    3:"1F60D",
    4:"1F60E",
    5:"1F62B",
    6:"1F62D",
    7:"1F64F",
    8:"1F494",
    9:"1F602",
    10:"1F609",
    11:"1F612",
    12:"1F614",
    13:"1F618",
    14:"1F629",
    15:"1F631",
    16:"1F633",
    17:"1F648",
    18:"2764"
}
uni_dict = {
    "1F44C": u"\U0001F44C",
    "1F44D": u"\U0001F44D",
    "1F60A": u"\U0001F60A",
    "1F60D": u"\U0001F60D",
    "1F60E": u"\U0001F60E",
    "1F62B": u"\U0001F62B",
    "1F62D": u"\U0001F62D",
    "1F64F": u"\U0001F64F",
    "1F494": u"\U0001F494",
    "1F602": u"\U0001F602",
    "1F609": u"\U0001F609",
    "1F612": u"\U0001F612",
    "1F614": u"\U0001F614",
    "1F618": u"\U0001F618",
    "1F629": u"\U0001F629",
    "1F631": u"\U0001F631",
    "1F633": u"\U0001F633",
    "1F648": u"\U0001F648",
    "2764":  u"\U00002764"
}

In [17]:
def pred_emoji(sample_sents, model):
    sent_len = 20
    sample_X = np.zeros((len(sample_sents), sent_len), dtype=np.int32)
    for i,sent in enumerate(sample_sents):
        sent_tokens = sent.strip().lower().split() #lazy tokenization
        sent_inds = [lookup_with_unk[s] if s in lookup_with_unk else UNK_IND for s in sent_tokens]
        sent_inds = sent_inds[:sent_len] #truncate if necessary
        sample_X[i, :len(sent_inds)] = sent_inds
    pred_prob =model.predict(sample_X)
    for k in range(len(sample_sents)):
        idx = np.argsort(pred_prob[k])[::-1]
        print("Sentence:{}".format(sample_sents[k]))
        for i in range(5):
            print("{}% {}".format(np.around(pred_prob[k][idx[i]]*100, decimals = 2), uni_dict[label_dict_r[idx[i]]]))        

In [19]:
model_0 = load_model(0)
model_1 = load_model(1)
model_4 = load_model(2)
model_5 = load_model(3)


Loaded model0 from disk
Loaded model1 from disk
Loaded model2 from disk
Loaded model3 from disk


In [20]:
sample_sents= [
    "Well done!",
    "I love you.",
    "This is bad.",
    "Seattle seahawk is doing great!",
    "Washington Wizards sucks this season."
]

In [21]:
print("Model 0")
pred_emoji(sample_sents, model_0)
print("Model 1")
pred_emoji(sample_sents, model_1)
print("Model 4")
pred_emoji(sample_sents, model_4)
print("Model 5")
pred_emoji(sample_sents, model_5)

Model 0
Sentence:Well done!
14.29% 😘
11.9% 🙏
11.04% 😔
9.83% 😎
8.94% 👍
Sentence:I love you.
36.44% 😘
9.85% 😔
9.77% 😍
7.14% 💔
6.98% 🙏
Sentence:This is bad.
10.33% 😔
8.18% 😍
8.03% 😭
7.62% 😎
7.51% 🙏
Sentence:Seattle seahawk is doing great!
12.61% 😎
9.7% 👌
8.89% 👍
8.2% 🙏
7.24% 😒
Sentence:Washington Wizards sucks this season.
17.42% 💔
16.62% 😔
14.68% 😒
11.19% 😩
10.74% 😫
Model 1
Sentence:Well done!
35.0% 😘
10.61% 😔
10.32% 🙏
7.14% 😂
5.85% 😒
Sentence:I love you.
34.21% 💔
26.57% 😔
7.79% 😘
5.65% 😍
5.62% 😩
Sentence:This is bad.
16.83% 😍
11.96% 😩
11.7% 😫
8.62% 😭
8.17% 💔
Sentence:Seattle seahawk is doing great!
29.88% 😎
12.67% 😒
9.52% 😩
7.21% 😫
6.92% 👌
Sentence:Washington Wizards sucks this season.
29.3% 😒
18.06% 😫
15.28% 😩
14.85% 😔
9.98% 😭
Model 4
Sentence:Well done!
13.68% 😘
9.74% 😔
9.32% 🙏
9.25% 😊
8.83% 👍
Sentence:I love you.
44.04% 😘
11.05% 😔
10.7% 😍
10.3% 🙏
7.4% 💔
Sentence:This is bad.
9.84% 💔
9.39% 👌
9.2% 😒
8.82% 😍
8.78% 😭
Sentence:Seattle seahawk is doing great!
14.55% 😎
12.35% 😒
10.34% 👌
7.7

In [22]:
#Impact of Tone of Voice
sample_sents= [
    "You love hurting me, huh?",
    "This is the shit!",
    "My flight is delayed.. amazing.",
    "That's just what I need, great! Terrific!",
    "Well what a surprise!",
    "That's just what we need!"       
]

In [23]:
print("Model 0")
pred_emoji(sample_sents, model_0)
print("Model 1")
pred_emoji(sample_sents, model_1)
print("Model 4")
pred_emoji(sample_sents, model_4)
print("Model 5")
pred_emoji(sample_sents, model_5)

Model 0
Sentence:You love hurting me, huh?
26.97% 😘
17.49% 💔
11.48% 😔
7.17% 😍
5.89% 😭
Sentence:This is the shit!
9.33% 😭
8.86% 😫
8.59% 😍
7.8% 😎
7.78% 👌
Sentence:My flight is delayed.. amazing.
10.91% 😔
8.3% 💔
7.95% 😎
7.81% 🙏
7.52% 😩
Sentence:That's just what I need, great! Terrific!
10.79% 💔
9.79% 😔
8.48% 😉
8.42% 😫
6.85% 😭
Sentence:Well what a surprise!
13.73% 😉
12.39% 👍
10.51% 😎
8.22% 😊
7.69% 😘
Sentence:That's just what we need!
12.31% 😔
9.98% 💔
8.46% 😉
8.25% 😎
7.25% 😂
Model 1
Sentence:You love hurting me, huh?
53.13% 💔
17.14% 😔
9.14% 😘
4.6% 😭
2.72% 😫
Sentence:This is the shit!
15.35% 😍
12.78% 😫
12.46% 😩
8.51% 😭
8.34% 😒
Sentence:My flight is delayed.. amazing.
14.65% 😍
14.53% 😎
10.63% 😩
9.62% 😫
8.64% 😊
Sentence:That's just what I need, great! Terrific!
13.54% 😉
9.17% 😒
8.07% 😎
7.92% 😊
7.4% 💔
Sentence:Well what a surprise!
20.85% 😉
9.69% 😎
9.68% 😊
8.16% 👍
7.86% 😒
Sentence:That's just what we need!
14.83% 😉
12.35% 😒
9.31% 😎
7.34% 💔
7.19% 😊
Model 4
Sentence:You love hurting me, huh?
30.1

In [24]:
sample_sents= [
    "Honesty is the best policy -- when there is money in it.", # - Mark Twain
    "You see, money's not everything in life is it? But it keeps you in touch with your children...", #- Johnnie Casson
    "Sometimes I need what only you can provide: your absence.", # - Ashleigh Brilliant
    "I never forget a face, but in your case I'll be glad to make an exception.", # - Groucho Marx
    "Zombies only eat brains; don't worry your safe!"
]

In [25]:
print("Model 0")
pred_emoji(sample_sents, model_0)
print("Model 1")
pred_emoji(sample_sents, model_1)
print("Model 4")
pred_emoji(sample_sents, model_4)
print("Model 5")
pred_emoji(sample_sents, model_5)

Model 0
Sentence:Honesty is the best policy -- when there is money in it.
16.44% 👍
13.83% 👌
12.49% 😉
10.84% 😎
7.03% 😂
Sentence:You see, money's not everything in life is it? But it keeps you in touch with your children...
13.32% 😉
12.78% 😊
9.44% 💔
9.16% 🙏
7.33% 👍
Sentence:Sometimes I need what only you can provide: your absence.
11.3% 💔
11.28% 😔
9.51% 😭
9.2% 😫
8.26% 😩
Sentence:I never forget a face, but in your case I'll be glad to make an exception.
15.31% 😉
15.18% 😊
13.46% 🙏
12.46% 👍
6.83% 😘
Sentence:Zombies only eat brains; don't worry your safe!
17.4% 😉
13.74% 👍
13.03% 😎
9.03% 😊
8.79% 👌
Model 1
Sentence:Honesty is the best policy -- when there is money in it.
20.92% 👌
15.35% 👍
10.74% 😉
10.59% 😊
9.33% 😎
Sentence:You see, money's not everything in life is it? But it keeps you in touch with your children...
24.06% 😉
11.25% 😊
9.47% 😘
9.18% 👌
7.54% 😎
Sentence:Sometimes I need what only you can provide: your absence.
23.22% 💔
20.5% 😔
17.26% 😒
8.83% 😩
8.07% 😫
Sentence:I never forget a fac

In [29]:
plot_model(model_4, to_file='model_4.png')
plot_model(model_5, to_file='model_5.png')